In [1]:
#模拟登陆
'''目前比较流行的实现方法有两种，一个是基于session和cookie的验证，一种是基于jwt（json web token）
的验证'''

#session就是存在服务端的，保存了用户此次访问的会话信息。
#cookie则是保存在用户本地浏览器的，它会在每次用户访问网站的时候作为请求头发送给服务器，
#服务器会从中找出session对象并判断起访问用户的状态最后返回response内容
#方法一，把cookie放到代码中，每次爬虫请求时再把它放到请求头中
#方法二，如果不想有任何手工操作，就直接用爬虫模拟登陆过程，即post请求，直接爬虫提交用户名
#密码给服务器，服务器放回的response headers里面可能会有set cookie字段，只需把这些cookie保存下来
#所以这个过程最重要的是维持好cookie。常遇到的问题，登陆过程伴随着各种校验参数，不好直接模拟请求；
#网站设置cookie的时候是通过javascript实现，所以需要仔细分析其中逻辑，尤其是用requests这样的请求库
#进行模拟登陆的时候，遇到的问题经常比较多
#方法三，用selenium，puppeteer或playwright来实现登陆过程的自动化
#总之，每次请求都携带好cookie信息就能实现模拟登陆里




'目前比较流行的实现方法有两种，一个是基于session和cookie的验证，一种是基于jwt（json web token）\n的验证'

In [2]:
#打开开发者工具，观察账号密码登陆时服务器发生的请求。
#登陆的瞬间，浏览器发起了一个post请求，目标URL为https://login2.scrape.center/login
#同时还提交了用户名和密码这两个数据。提交完成后，返回状态码为302，同时在set cookie里面有session ID

import requests
from urllib.parse import urljoin

BASE_URL = 'https://login2.scrape.center/'
LOGIN_URL = urljoin(BASE_URL, '/login')
INDEX_URL = urljoin(BASE_URL, '/page/1')
USERNAME = 'admin'
PASSWORD = 'admin'

response_login = requests.post(LOGIN_URL, data={
    'username': USERNAME,
    'password': PASSWORD
}, allow_redirects=False)#把重定向关掉
print(type(response_login))#是个response类

cookies = response_login.cookies#获取其cookie信息
print('Cookies', cookies)

response_index = requests.get(INDEX_URL, cookies=cookies)
print('Response Status', response_index.status_code)
print('Response URL', response_index.url)

<class 'requests.models.Response'>
Cookies <RequestsCookieJar[<Cookie sessionid=j6c1h3qrll49i78bnzo3ceugdvnuvjx3 for login2.scrape.center/>]>
Response Status 200
Response URL https://login2.scrape.center/page/1


In [3]:
#如果想每次都不用带着登陆时获取的cookie请求，来请求接下来的每个页面，可以直接用session
#直接用session类代替requests
import requests
from urllib.parse import urljoin

BASE_URL = 'https://login2.scrape.center/'
LOGIN_URL = urljoin(BASE_URL, '/login')
INDEX_URL = urljoin(BASE_URL, '/page/1')
USERNAME = 'admin'
PASSWORD = 'admin'

session = requests.Session()

response_login = session.post(LOGIN_URL, data={
    'username': USERNAME,
    'password': PASSWORD
})

cookies = session.cookies
print('Cookies', cookies)

response_index = session.get(INDEX_URL)
print('Response Status', response_index.status_code)
print('Response URL', response_index.url)


Cookies <RequestsCookieJar[<Cookie sessionid=b38k9h476r1t93jj7ryo0sfuwq5ag0i1 for login2.scrape.center/>]>
Response Status 200
Response URL https://login2.scrape.center/page/1


In [2]:
#如果碰到难以模拟登陆的过程（比如带有验证码，带有加密参数等情况），就可以用selenium等模拟登陆目的是获取登陆后的cookie，
#获取到cookie后，和前面一样用这些cookie爬取其他页面就好



from urllib.parse import urljoin
from selenium import webdriver
import requests
import time

BASE_URL = 'https://login2.scrape.center/'
LOGIN_URL = urljoin(BASE_URL, '/login')
INDEX_URL = urljoin(BASE_URL, '/page/1')
USERNAME = 'admin'
PASSWORD = 'admin'

browser = webdriver.Chrome()#用selenium先打开了chrome
browser.get(BASE_URL)#跳转到登陆页面。下面是模拟输入了用户名和密码，
browser.find_element_by_css_selector('input[name="username"]').send_keys(USERNAME)
browser.find_element_by_css_selector('input[name="password"]').send_keys(PASSWORD)
browser.find_element_by_css_selector('input[type="submit"]').click()#这里是点击了登录按钮
time.sleep(10)

# get cookies from selenium
cookies = browser.get_cookies()#获取当前浏览器的所有cookie。用这些就能访问其他数据了
print('Cookies', cookies)
browser.close()

# set cookies to requests
session = requests.Session()#声明了requests的session对象
for cookie in cookies:#遍历刚才的cookie并将其设置到session对象的cookie属性上
    session.cookies.set(cookie['name'], cookie['value'])

response_index = session.get(INDEX_URL)#接着再拿这个session对象去请求INDEX_URL，就能够直接获取页面而不会跳转到登陆页面了
print('Response Status', response_index.status_code)
print('Response URL', response_index.url)

AttributeError: 'WebDriver' object has no attribute 'find_element_by_css_selector'

In [ ]:
#多线程学习https://cuiqingcai.com/202271.html


In [ ]:
#账号池
#如果爬虫要求爬取的数据量比较大或爬取速度比较快，而网站又有单账号并发限制或访问状态检测
#等反爬虫手段，那我们等账号可能就会无法访问或者面临封号分险
#这种情况下，我们可以使用分流等方案来实现。假设某个网站设置一分钟之内检测到同一个账号访问
#三次获三次以上则封号。我们就可以建立一个账号池，用多个账号来随机访问或爬取数据，这样就能大幅度提高
#爬虫的并发量，降低被封号的风险。
#比如我们可以准备100个账号，然后100个账号都模拟登陆，把对应的cookie或者jwt存下来，每次访问的时候
#随机取一个，由于账号多，所以每个账号被取用的概率也就降下来了，也就降低封号风险